## INTRODUCCIÓN
Nos proponemos estudiar los datos acerca del sistema de alquiler de bicicletas de la ciudad de San Francisco. Para ello contamos con sets de datos que contienen información sobre las estaciones, estado de cada estación minuto a minuto, estado del clima y datos de cada viaje.


### Buscamos obtener la siguiente informacion
- Viajes con mayor y menor duracion
- Cantidad total de viajes por ciudad y por estacion
- Ubicaciones de las estaciones, influye la ubicacion de las estaciones con la cantidad de viajes produciadas ?
- Relacion entre las viajes y el clima
- Estaciones mas usadas 
- Uso de las bicicletas en dias festivos

Link al repo: https://github.com/emeee/bikes7506

In [ ]:
##Imports & Reads
%matplotlib inline

import pandas as pd
import numpy as np
import datetime as datetime
import matplotlib.pyplot as plt

plt.rcParams['figure.figsize'] = (15, 5)

In [ ]:
trip = pd.read_csv("../input/trip.csv")
weather = pd.read_csv('..input/weather.csv')
station = pd.read_csv('../input/station.csv')

In [ ]:
# Creamos un nuevo df con los zipcodes de cada ciudad para mergear con station

cities = ['San Francisco', 'Redwood City', 'Palo Alto', 'Mountain View', 'San Jose']
zips = [94107, 94063, 94301, 94041, 95113]
data = {'zip':zips, 'city':cities}
zipcode = pd.DataFrame(data=data)
estaciones = pd.merge(station, zipcode, how='inner')

In [ ]:
##Comprobacion de tipos // paso a datetime solo el dia
trip.dtypes

In [ ]:
trip['start_date'] = pd.to_datetime(trip['start_date'])
trip['end_date'] = pd.to_datetime(trip['end_date'])

In [ ]:
weather.dtypes

In [ ]:
weather['date'] = pd.to_datetime(weather['date'])

In [ ]:
trip['date'] = pd.to_datetime(trip['start_date'].dt.date)
trip['end_date'] = pd.to_datetime(trip['end_date'].dt.date)

In [ ]:
trip['id'] = trip['start_station_id']

In [ ]:
viajes = pd.merge(trip,estaciones,on='id', how='right')

In [ ]:
viajes['zip_code'] = viajes['zip']

In [ ]:
completo = pd.merge(viajes, weather, how='inner')

In [ ]:
completo['duration'] = completo['duration']/60
weather['day'] = weather['date'].map(lambda x: x.strftime('%d/%m'))
weather['year'] = weather['date'].map(lambda x: x.strftime('%y'))

# Analisis de los viajes en relación al clima

In [ ]:
# Menor temperatura
weather[weather['max_temperature_f'] == weather['max_temperature_f'].min()]

In [ ]:
# Mayor temperatura
weather[weather['max_temperature_f'] == weather['max_temperature_f'].max()]

In [ ]:
#dias mas calurosos de 2014
weather[weather['year'] == '14'].groupby('day').mean()['max_temperature_f'].sort_values(ascending = False).head(30).plot(kind = 'bar', figsize=(14,4));



In [ ]:


completo[completo.duration < 360].plot.scatter('mean_temperature_f', 'duration', alpha=0.25, title='Duracion del viaje segun la temperatura')
sub1 = completo[completo.duration < 360]
sub1[sub1.subscription_type == "Subscriber"].plot.scatter('mean_temperature_f', 'duration', alpha=0.25, color="red", title='Subscribers')
sub1[sub1.subscription_type == "Customer"].plot.scatter('mean_temperature_f','duration', alpha=0.25, color="green", title='Customers')

In [ ]:
#Cuando la temperatura es promedio los viajes duran mucho mas en promedio
graf = completo[completo.duration < 360]
graf.groupby('mean_temperature_f').mean()['duration'].plot(title='Tiempo de viaje promedio segun la temperatura - Azul: Todos - Verde: Customers - Naranja: Subscribers')
graf[graf.subscription_type == 'Subscriber'].groupby('mean_temperature_f').mean()['duration'].plot()
graf[graf.subscription_type == 'Customer'].groupby('mean_temperature_f').mean()['duration'].plot()

In [ ]:
#Segun los graficos anteriores podemos deducir que en T>80 hay pocos viajes y no siguen el patron por lo tanto grafico sacandolos

graf = completo[completo.duration < 360]
graf = graf[graf.mean_temperature_f < 80]
graf.groupby('mean_temperature_f').mean()['duration'].plot(title='Tiempo de viaje promedio segun la temperatura - Azul: Todos - Verde: Customers - Naranja: Subscribers')
graf[graf.subscription_type == 'Subscriber'].groupby('mean_temperature_f').mean()['duration'].plot()
graf[graf.subscription_type == 'Customer'].groupby('mean_temperature_f').mean()['duration'].plot()

In [ ]:
#Cantidad de viajes segun temperatura promedio (puede ser que haya muchos mas dias de temperatura promedio)
graf = completo[completo.duration < 360]
graf[graf.duration < 360].groupby('mean_temperature_f').count()['duration'].plot('area', alpha=0.3, title='Cantidad de viajes netos segun temperatura promedio - Azul: Todos - Verde: Customers - Naranja: Subscribers')
graf[graf.subscription_type=='Subscriber'].groupby('mean_temperature_f').count()['duration'].plot()
graf[graf.subscription_type=='Customer'].groupby('mean_temperature_f').count()['duration'].plot()

In [ ]:
#Cantidad de viajes promedio
cant_dias = weather.groupby('mean_temperature_f').count()['date']
graf = completo[completo.duration < 360]
g1 = graf.groupby('mean_temperature_f').count()['duration']/cant_dias
g1.plot('area', alpha=0.3, title='Cantidad de viajes promedio segun temperatura - Azul: Todos - Verde: Customers - Naranja: Subscribers')
g2 = graf[graf.subscription_type == 'Subscriber'].groupby('mean_temperature_f').count()['duration']/cant_dias
g2.plot()
g3 = graf[graf.subscription_type == 'Customer'].groupby('mean_temperature_f').count()['duration']/cant_dias
g3.plot()

In [ ]:
#A medida que aumenta la visibilidad aumentan los viajes netos (hay muchos mas dias con visibilidad 10)
completo.groupby('mean_visibility_miles').count()['duration'].plot('area',alpha=0.25,title='Cantidad de viajes netos segun visibilidad - Naranja: Cantidad de dias con esa visibilidad * 100')
cant_dias = weather.groupby('mean_visibility_miles').count()['date']
cant_dias = cant_dias*100
cant_dias.plot()
#En naranja la cantidad de dias

In [ ]:
#En promedio no varia tanto la visiblidadad con la cantidad de viajes pero por alguna razon arriba de las 10 millas cae de repente

cant_dias = weather.groupby('mean_visibility_miles').count()['date']
g1 = completo.groupby('mean_visibility_miles').count()
g1['duration'] = g1['duration']/cant_dias
g1['duration'].plot('area',alpha=0.25, title='Cantidad de viajes promedio segun visibilidad - Azul: Total - Naranja: Subscribers - Verde: Customers')

g2 = completo[completo.subscription_type == 'Subscriber'].groupby('mean_visibility_miles').count()['duration']/cant_dias
g2.plot()


g3 = completo[completo.subscription_type == 'Customer'].groupby('mean_visibility_miles').count()['duration']/cant_dias
g3.plot()

In [ ]:
#Cantidad de viajes netos segun la velocidad del viento (probablemente haya muchos con velocidad 10 mph)
completo[completo.duration < 360].groupby('mean_wind_speed_mph').count()['duration'].plot('area', alpha=0.25)
cant_dias = weather.groupby('mean_wind_speed_mph').count()['date']
#Para que este en escala y se vea en el grafico multiplico por 100
(cant_dias*100).plot(title='Area: Cantidad de viajes netos segun velocidad del viento  -  Naranja: Cantidad de dias con esa velocidad * 100')

In [ ]:
#Cuanto mas viento hay mas se anda en bici promedio

cant_dias = weather.groupby('mean_wind_speed_mph').count()['date']
g1 = completo.groupby('mean_wind_speed_mph').count()
g1.duration = g1.duration/cant_dias
g1.duration.plot('area',alpha=0.25, title='Cantidad de viajes promedio segun velocidad dle viento - Azul: Todos - Verde: Customers - Naranja: Subscribers')

g2 = completo[completo.subscription_type == 'Subscriber'].groupby('mean_wind_speed_mph').count()['duration']/cant_dias
g2.plot()


g3 = completo[completo.subscription_type == 'Customer'].groupby('mean_wind_speed_mph').count()['duration']/cant_dias
g3.plot()

# Analisis de viajes por estaciones

NOTA! El codigo que genera los siguiente mapas se encuentra en nuestro repositorio de github (link en la introduccion) 

## Mapa con todas las estaciones 

![Mapa estaciones san francisco][1]


  [1]: http://i.imgur.com/sspNx46.png

In [ ]:
# Duracion promedio
(trip['duration'].sum() / trip['duration'].size) / 60

In [ ]:
trip['duration'].sort_values(ascending=False).reset_index().head(10)

Como vemos que las maximas duraciones son numeros que representan varios meses, suponemos que son datos mal cargados o bicis que se olvidaron se devolver. Esto afecta claramente al promedio sacado anteriormente, por lo que decidimos calcularlo como la mayor frecuencia de una duracion

In [ ]:
max_durations = trip.groupby(by='duration').count().sort_values(ascending=False, by='id')[[0]].head(10)
max_durations.rename(columns={'id': 'Total'})

Aca podemos observar que la duracion que mas frecuencia tiene es 359 segundos, que son aproximadamente 6 minutos

In [ ]:
#Menores duraciones
trip['duration'].sort_values().reset_index().head(10)

In [ ]:

trips = []

for row in trip:
    trips.append(trip.start_date.dt.hour + (trip.start_date.dt.minute)/60)
    


In [ ]:
trip['time'] = trip['start_date'].map(lambda x: x.strftime('%H'))
trip['day'] = trip['start_date'].map(lambda x: x.strftime('%d/%m'))
trip['year'] = trip['start_date'].map(lambda x: x.strftime('%y'))


### Total de viajes por ciudad y en dias festivos

In [ ]:
n, bins, patches = plt.hist(trips, bins=24, facecolor='blue', alpha=0.65, histtype='stepfilled')
plt.xlabel('Horas del dia')
plt.ylabel('Cantidad de viajes')
plt.title('Histograma de viajes en un dia ')
plt.axis([0, 24, 0, 90000]) #Rangos
plt.grid(True)

plt.show()

In [ ]:
#31 de diciembre
plt.ylabel('trips')
trip[(trip['day'] == '31/12') & (trip['year'] == '13')].groupby('time').size().plot(kind = 'bar',figsize = (14,4))


In [ ]:
#1 de enero (año nuevo)
plt.ylabel('trips')
trip[(trip['day'] == '01/01') & (trip['year'] == '14')].groupby('time').size().plot(kind = 'bar',figsize = (14,4))


In [ ]:
#3 de julio 
plt.ylabel('trips')
trip[trip['day'] == '03/07'].groupby('time').size().plot(kind = 'bar',figsize = (14,4))

In [ ]:
#4 de julio (dia de la independencia)
plt.ylabel('Trips')
trip[trip['day'] == '04/07'].groupby('time').size().plot(kind = 'bar',figsize = (14,4))

In [ ]:
#24 de diciembre
plt.ylabel('trips')
trip[trip['day'] == '24/12'].groupby('time').size().plot(kind = 'bar',figsize = (14,4))

In [ ]:
#25 de diciembre (navidad)
plt.ylabel('trips')
trip[trip['day'] == '25/12'].groupby('time').size().plot(kind = 'bar',figsize = (14,4))

In [ ]:
#Mes halloween (cae el 31)
trip[(trip['month'] == '10') & (trip['year'] == '13')].groupby('day').size().plot(kind = 'bar',figsize = (14,4))


In [ ]:
#Mes siguiente a halloween
trip[(trip['month'] == '11') & (trip['year'] == '13')].groupby('day').size().plot(kind = 'bar',figsize = (14,4))


In [ ]:
plt.title('Cantidad de viajes por ciudad', color='black')
plt.ylabel('Cantidad de viajes')
viajes.groupby(by='city')['city'].count().plot(kind='bar')

### Cantidad de viajes por estacion en la ciudad de San Francisco

In [ ]:
san_francisco = viajes[viajes['city']== 'San Francisco']
plt.title('Cantidad de viajes en San Francisco', color='black')
plt.ylabel('Cantidad de viajes')
san_francisco.groupby(by='start_station_name')['start_station_name'].count().plot(kind='bar', figsize=(14,4))

### Top 10 estaciones mas usadas en la ciudad de San Francisco

In [ ]:
plt.title('10 estaciones mas usadas en San Francisco', color='black')
plt.ylabel('Cantidad de viajes')
san_francisco.groupby(by='start_station_name')['start_station_name'].count().sort_values(ascending=False).head(10).plot(kind='bar', figsize=(14,4))

### Mapa con las 10 estaciones mas usadas de san francisco 

![Estaciones mas usadas en San Francisco][1]


  [1]: http://i.imgur.com/hQdY3Yc.png

###Cantidad de viajes por estacion en San Jose

In [ ]:
san_jose = viajes[viajes['city']== 'San Jose']
plt.title('Cantidad de viajes en San Jose', color='black')
plt.ylabel('Cantidad de viajes')
san_jose.groupby(by='start_station_name')['start_station_name'].count().plot(kind='bar', figsize=(14,4))

### Cantidad de viajes por estacion en Palo Alto

In [ ]:
palo_alto = viajes[viajes['city']== 'Palo Alto']
plt.title('Cantidad de viajes en Palo Alto', color='black')
plt.ylabel('Cantidad de viajes')
palo_alto.groupby(by='start_station_name')['start_station_name'].count().plot(kind='bar', figsize=(14,4))

### Cantidad de viajes por estacion en Mountain View

In [ ]:
mountain_view = viajes[viajes['city']== 'Mountain View']
plt.title('Cantidad de viajes en Mountain View', color='black')
plt.ylabel('Cantidad de viajes')
mountain_view.groupby(by='start_station_name')['start_station_name'].count().plot(kind='bar', figsize=(14,4))

### Cantidad de viajes por estacion en Redwood City

In [ ]:
redwood = viajes[viajes['city']== 'Redwood City']
plt.title('Cantidad de viajes en  Redwood City', color='black')
plt.ylabel('Cantidad de viajes')
redwood.groupby(by='start_station_name')['start_station_name'].count().plot(kind='bar', figsize=(14,4))

In [ ]:
####################

In [ ]:
trip['day'] = trip['start_date'].map(lambda x: x.strftime('%d/%m'))

In [ ]:
#cantidad alquiles 2013
trip[trip['year'] == '13'].groupby('day').size().sort_values(ascending = False).head(30).plot(kind = 'bar',figsize = (14,4))


In [ ]:
#cantidad alquileres 2015
trip[trip['year'] == '15'].groupby('day').size().sort_values(ascending = False).head(30).plot(kind = 'bar',figsize = (14,4))